In [49]:

# Libraries dependencies
import MetaTrader5 as mt5
from datetime import datetime, timezone
import pandas as pd
import numpy as np

# Project files
from AlgorithmicTrading.models.metatrader import (
    MqlTradeRequest, 
    MqlPositionInfo, 
    MqlAccountInfo, 
    MqlTradeOrder, 
    MqlTick, 
    MqlSymbolInfo,   
    ENUM_ORDER_TYPE_MARKET,
    ENUM_POSITION_TYPE, 
    ENUM_DEAL_ENTRY,
    ENUM_DEAL_TYPE, 
    ENUM_TIMEFRAME, 
    ENUM_ACCOUNT_MARGIN_MODE
)
from AlgorithmicTrading.trade.trade import Trade
from AlgorithmicTrading.account import AccountBacktest, AccountLive
from AlgorithmicTrading.rates import Rates
from AlgorithmicTrading.utils.trades import get_last_tick


In [50]:
# Hedge account
# account = AccountLive.login(login=75181006, server="MetaQuotes-Demo", password="MfK*K1Vc")
# Netting account
account = AccountLive.login(login=75185597, server="MetaQuotes-Demo", password="@cBjZ0Tp")
# EUR account
# account = AccountLive.login(login=75180148, server="MetaQuotes-Demo", password="Iq-g2cFm")
# Backtest account
# account = AccountBacktest.login(balance=100_000, margin_mode=ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING)
account

[INFO]: Successfull login to #75185597 account


MqlAccountInfo(login=75185597, trade_mode=<ENUM_ACCOUNT_TRADE_MODE.ACCOUNT_TRADE_MODE_DEMO: 0>, leverage=100, limit_orders=100, margin_so_mode=<ENUM_ACCOUNT_STOPOUT_MODE.ACCOUNT_STOPOUT_MODE_PERCENT: 0>, trade_allowed=True, trade_expert=True, margin_mode=<ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING: 0>, currency_digits=2, fifo_close=False, balance=49555.55000000002, credit=0.0, profit=-780.56, equity=48773.79, margin=49519.63, margin_free=-745.84, margin_level=98.49384981269044, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Pedro Bento', server='MetaQuotes-Demo', currency='USD', company='MetaQuotes Software Corp.', orders=[], positions=[MqlPositionInfo(ticket=2044088111, time=datetime.datetime(2023, 10, 20, 23, 59, tzinfo=datetime.timezone.utc), time_msc=datetime.datetime(2023, 10, 20, 23, 59, 0, 561000, tzinfo=datetime.timezone.utc), time_update=datetime.datetime(2023, 10, 

In [51]:
n_rates = Rates.get_candles_range(
    "USDJPY", 
    date_from=datetime(2022, 9, 1, tzinfo=timezone.utc), 
    date_to=datetime(2022, 11, 30, tzinfo=timezone.utc), 
    timeframe=ENUM_TIMEFRAME.TIMEFRAME_M15
)

In [52]:
from AlgorithmicTrading.backtest.environment.environment import TradingEnv

n_rates = TradingEnv.set_col_names(
    n_rates.reset_index(), 
    adj_close_col_name="close",
    close_col_name="close",
    datetime_col_name="time",
    high_col_name="high",
    low_col_name="low",
    open_col_name="open",
    volume_col_name="tick_volume",
)
n_rates

,Open,High,Low,Close,Adj Close,Volume,Datetime
0,138.928,138.945,138.798,138.945,138.945,193,2022-09-01 00:00:00+00:00
1,138.945,139.016,138.945,139.007,139.007,1183,2022-09-01 00:15:00+00:00
2,139.006,139.038,138.999,139.026,139.026,255,2022-09-01 00:30:00+00:00
3,139.026,139.087,139.019,139.066,139.066,484,2022-09-01 00:45:00+00:00
4,139.060,139.136,139.014,139.084,139.084,1452,2022-09-01 01:00:00+00:00
...,...,...,...,...,...,...,...
6136,138.791,138.797,138.695,138.716,138.716,721,2022-11-29 23:00:00+00:00
6137,138.716,138.730,138.666,138.681,138.681,602,2022-11-29 23:15:00+00:00
6138,138.682,138.712,138.665,138.692,138.692,790,2022-11-29 23:30:00+00:00
6139,138.691,138.730,138.670,138.696,138.696,945,2022-11-29 23:45:00+00:00


In [53]:
from AlgorithmicTrading.backtest.environment.environment import TradingEnv

env = TradingEnv(n_rates)
env.reset()

[INFO]: Successfull backtest account create


({'trend_slope': 0.00660000000000025,
  'price_diff': -0.020000000000010232,
  'support_trend_line_slope': 0.02431692477817124,
  'resistance_trend_line_slope': 0.00043933736625319567,
  'volume': 870,
  'currently_holding': 0},
 {})

In [55]:

import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define the DQN agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Set state and action size
        self.state_size = state_size
        self.action_size = action_size
        
        # Define replay learning buffer
        self.replay_buffer = deque(maxlen=5000) 
        self.learning_rate = 0.001
        
        # Discount factor
        self.gamma = 0.95
        
        # Exploration rate
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        
        # Which we want to update the target network
        self.update_rate = 1000
        self.main_network = self.build_network()
        self.target_network = self.build_network()
        self.target_network.set_weights(self.main_network.get_weights())

        
    def build_network(self):
        main_network = Sequential()
        main_network.add(Dense(24, input_dim=self.state_size, activation='relu'))
        main_network.add(Dense(24, activation='relu'))
        main_network.add(Dense(self.action_size, activation='linear'))
        main_network.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return main_network

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.main_network.predict(state)
        return np.argmax(q_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))
        
    def epsilon_greedy(self, state):
        if random.uniform(0,1) < self.epsilon:
            return np.random.randint(self.action_size)
        Q_values = self.main_network.predict(state)
        return np.argmax(Q_values[0])

    def train(self, batch_size):
        if len(self.replay_buffer) < batch_size:
            return
        minibatch = random.sample(self.replay_buffer, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if not done:
                target_Q = (reward + self.gamma * np.amax(self.target_network.predict(next_state)))
            else:
                target_Q = reward

            Q_values = self.main_network.predict(state)
            # Update the target value
            Q_values[0][action] = target_Q
            
            # Train the main network
            self.main_network.fit(state, Q_values, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def update_target_network(self):
        self.target_network.set_weights(self.main_network.get_weights())


In [56]:
def preprocess_state(state):
    state = np.around(np.array(list(state.values())), 5)
    return state.reshape(1, len(state))

In [57]:
agent = DQNAgent(len(env.observation_space), env.action_space.n)

In [123]:
num_episodes = 2
num_timesteps = 500
batch_size = 10
num_past_candles = 4

done = False
time_step = 0

for i in range(num_episodes):
    Return = 0
    state = preprocess_state(env.reset()[0])

    for t in range(num_timesteps):
        # env.render()
        time_step += 1
        if time_step % agent.update_rate == 0:
            agent.update_target_network()
            
        action = agent.epsilon_greedy(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        Return += reward
        if t % 100 == 0:
            print("Timestep: ", t, time_step, ", Return", Return, agent.epsilon)
        next_state = preprocess_state(next_state)
        done = (terminated or truncated)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print('Episode: ',i, time_step,',' 'Return', Return)
            break


[INFO]: Successfull backtest account create
1/1 [==============================] - 0s 24ms/step
Timestep:  0 1 , Return -1.1 0.16291280823111093
1/1 [==============================] - 0s 20ms/step
Timestep:  100 101 , Return 51.7 0.16291280823111093
1/1 [==============================] - 0s 22ms/step
Timestep:  200 201 , Return -15.900000000000007 0.16291280823111093
1/1 [==============================] - 0s 21ms/step
Timestep:  300 301 , Return 3.3999999999999844 0.16291280823111093
1/1 [==============================] - 0s 22ms/step
Timestep:  400 401 , Return -73.60000000000002 0.16291280823111093
1/1 [==============================] - 0s 21ms/step
[INFO]: Successfull backtest account create
1/1 [==============================] - 0s 20ms/step
Timestep:  0 501 , Return -1.1 0.16291280823111093
1/1 [==============================] - 0s 22ms/step
Timestep:  100 601 , Return 34.20000000000002 0.16291280823111093
1/1 [==============================] - 0s 24ms/step
Timestep:  200 701 , Re

In [ ]:
if len(agent.replay_buffer) > batch_size:
    for i in range(100):
        print(i)
        agent.train(batch_size)

agent.epsilon

In [ ]:
# Rates.get_candles_range(
#     "EURUSD", 
#     date_from=datetime(2022, 10, 3, 16, tzinfo=timezone.utc),
#     date_to=datetime(2022, 10, 3, 20, tzinfo=timezone.utc),
#     timeframe=ENUM_TIMEFRAME.TIMEFRAME_H4
# )
# Rates.get_candles_before(
#     "EURUSD", 
#     date_to=datetime(2022, 10, 3, 20, tzinfo=timezone.utc),
#     timeframe=ENUM_TIMEFRAME.TIMEFRAME_H4,
#     n_candles=3,
# )
# mt5.copy_rates_range(    
#     "EURUSD", 
#     ENUM_TIMEFRAME.TIMEFRAME_H4,
#     datetime(2022, 10, 3, 16, tzinfo=timezone.utc),
#     datetime(2022, 10, 3, 20, tzinfo=timezone.utc),
# )
x=pd.DataFrame(mt5.copy_ticks_from(    
    "EURUSD",
    datetime(2022, 1, 3, 5, 8, 3, tzinfo=timezone.utc) ,
    100,
    mt5.COPY_TICKS_INFO,
))
x["time"] = pd.to_datetime(x.time, unit="s", utc=True)
x
# mt5.__dict__

In [ ]:
# state.shape
# episodes = 1000
# for i in 

In [ ]:
from collections import Counter
from datetime import timedelta

x = [timedelta(minutes=5), timedelta(minutes=5), timedelta(minutes=5), timedelta(minutes=6)]

Counter(x).most_common(2)

In [ ]:
AccountLive.get_data().positions[-1].__dict__


In [ ]:
account.history_deals[-1].magic = 2020

In [ ]:
trade_class = Trade(account, 101010, 2)
trade_class.sell(symbol="EURUSD", volume=0.2)

In [ ]:
import MetaTrader5 as mt5
mt5.TradeRequest


In [ ]:
trade_class.sell("USDJPY", volume=2)

In [ ]:
mt5.history_deals_get(
                datetime(2023, 6, 8, tzinfo=timezone.utc),
                datetime(2028, 6, 9, tzinfo=timezone.utc),
            )[-1]._asdict()

from datetime import timedelta
datetime.now(timezone.utc) + timedelta(days=2)

In [ ]:
account.history_deals

In [ ]:
account.history_deals[-1].dict()

In [ ]:
trade_class.close_position(account.positions[0].ticket)

In [ ]:
trade_class.modify_position(position=1686247346317113, stop_price=1, profit_price=2)

In [ ]:
# trade_class.account_data.positions[0]
account.positions[0].dict()

In [ ]:
trade_class.modify_pending_order(50477386568, 100, 149, 100)

In [ ]:
trade_class.modify_pending_order(account.orders[0].ticket, price=5, stop_price=3, profit_price=7, expiration=datetime(2023,1,1))

In [ ]:
deals = pd.DataFrame([deal.dict() for deal in account.history_deals])
deals["type"] = deals.type.apply(lambda value: ENUM_DEAL_TYPE(value).name)
deals["entry"] = deals.entry.apply(lambda value: ENUM_DEAL_ENTRY(value).name)
features = deals[["ticket", "time", "entry", "type", "position_id", "volume", "price", "profit"]]
features

In [ ]:
data = [deal.dict() for deal in account.history_deals]
pd.DataFrame(data)

In [ ]:
class a:
    x = 1
    
q = a()
w = a()

w.x = 3

e = [q,w]

e.index(w)

In [ ]:
a = Rates.get_last_n_candles("EURUSD")

In [ ]:
a["mean"] = a.close.rolling(14).mean()
a

In [ ]:
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.volume import AccDistIndexIndicator

import plotly.graph_objects as go

indicator_rsi_14 = AccDistIndexIndicator(
            high=a.high,
            low=a.low,
            close=a.close,
            volume=a.tick_volume,
        )

indicator_rsi_14.acc_dist_index()